In [1]:
import os
os.chdir('F:\\nanopore_data\\telomere\\AF16_extract')
import pandas as pd
from Bio import SeqIO
from Bio import Seq

In [ ]:
#IL
for stage in stg:
    position = "IL"
"""
stopped  here
"""
    anchor = "X:17708805-17718805"
    #file name module
    read_dict_fh = str("fasta/%s_%s.fa" % (stage,position))
    align_info_fh = str("blastn/%s_%s_direction.csv" % (stage,position))
    read_list_fh = str("list_filter/%s_%s_list" % (stage,position))
    output_fh = str("trimmed/%s_%s_read.fa" % (stage,position))
    #read inputed for consistant coding direction
    read_dict = SeqIO.index(read_dict_fh, "fasta")
    align_info = pd.read_csv(align_info_fh, sep = "\t",header=None,index_col=False)
    align_info.columns = ["query", "subject", "identity", "length", "mis","gap","qstart","qend","sstart","send","evalue","bit","strand"]
    with open(output_fh,"w") as out_w,open(read_list_fh,"r") as read_list :
        for line in read_list:
            readName = line.rstrip()
            read_seq = str(read_dict[readName].seq)
            df_tmp = align_info[(align_info["subject"]== readName)&\
                                (align_info["length"] > 500)&\
                                (align_info["query"] == anchor)&\
                                (align_info["qend"] > 9900)]
            #print(stage, readName)
            if  str(df_tmp.iloc[0,-1]) == "plus" :
                telomere_end = df_tmp["send"].max()
                out_seq = read_seq[telomere_end:]
                if len(out_seq) < 10:
                    continue
                else:
                    out_w.write(str(">%s\n" %(readName)))
                    out_w.write(out_seq+"\n")
            else:
                telomere_end = df_tmp["send"].min()
                out_seq = Seq.reverse_complement(read_seq[:telomere_end-1])
                if len(out_seq) < 10:
                    continue
                else:
                    out_w.write(str(">%s\n" %(readName)))
                    out_w.write(out_seq+"\n")
    out_w.close()
    read_list.close()